In [ ]:
# | default_exp utils.mission
# | export
import yaml
import polars as pl
from space_analysis.core import Mission, Instrument, InstrumentSuite
from pathlib import Path

In [ ]:
# | export
def parse_mission_yaml(file: Path):
    with open(file, "r") as stream:
        data = yaml.safe_load(stream)
    return Mission(**data)


def parse_missions_yaml(dir: Path, suffixs=[".yaml", ".yml"]):
    """Parse the mission yaml files in the given directory"""
    return [parse_mission_yaml(f) for f in dir.iterdir() if f.suffix in suffixs]

In [ ]:
# | export
# Recursive function to find magnetometers
def find_instrument(instruments: list[Instrument | InstrumentSuite], ins_type):
    ins_suites = [i for i in instruments if isinstance(i, InstrumentSuite)]
    ins_in_suite = [i for suite in ins_suites for i in suite.instruments]
    ins_out_suite = [i for i in instruments if isinstance(i, Instrument)]

    return [i for i in ins_in_suite + ins_out_suite if i.type == ins_type]


def parse_missions(data: list[Mission], info, ins_type=None):
    rows = []

    for mission in data:
        if ins_type is None:
            rows.append([mission.name, mission.model_dump(warnings="none").get(info)])
            continue
        instruments = find_instrument(mission.instruments, ins_type)
        for i in instruments:
            rows.append([mission.name, i.model_dump(warnings="none").get(info)])

    return pl.DataFrame(rows, schema=["Mission", info], orient="row")


# Define the merge function
def merge_dfs(left: pl.DataFrame, right, on="Mission", how="full", coalesce=True):
    return left.join(right, on=on, how=how, coalesce=coalesce)

In [ ]:
from beforerr.project import datadir
from functools import reduce
from great_tables import GT

In [ ]:
dir = datadir() / "missions"
missions = parse_missions_yaml(dir)

In [ ]:
# | label: tbl-missions
# | tbl-cap: Missions info
# Parse and display the table
df_mag = parse_missions(missions, "time_resolutions", "magnetometer").rename(
    {"time_resolutions": "δt(B)"}
)
df_plasma = parse_missions(missions, "time_resolutions", "plasma").rename(
    {"time_resolutions": "δt(plasma)"}
)
df_r = parse_missions(missions, "radial_coverage")
df_time = parse_missions(missions, "launch_date")
df_link = parse_missions(missions, "website")

df_list = [df_mag, df_plasma, df_r, df_time, df_link]  # Replace with your DataFrames

df_merged = reduce(merge_dfs, df_list).with_columns(pl.col("website").list.join(", "))

GT(df_merged).fmt_markdown("website").cols_label(
    radial_coverage="Radial coverage",
    launch_date="Launch date",
)

Mission,δt(B),δt(plasma),Radial coverage,Launch date,website
STEREO,8 Hz,1 min,None,2006-10-26,None
WIND,11 Hz,1 Hz,None,1994-11-01,NASA
Parker Solar Probe,~200 Hz,0.25-1 Hz,0.05 - 1 AU,2018-08-12,NASA
ARTEMIS,5 Hz,0.25 Hz,None,2007-02-17,None
Juno,1 Hz,None,1 - 5.5 AU,2011-08-05,NASA
Solar Orbiter,None,None,0.28 - 0.9 AU,2020-02-10,ESA
